### Import Required Libraries

In [6]:
"""
Explainable Image Classification on CIFAR-10

This notebook implements the complete pipeline for 
1. Model Selection and Training (ResNet-50)
2. Data Preprocessing (CIFAR-10)
3. Fine-Tuning with Transfer Learning
4. XAI Explanations (Grad-CAM and LIME)
"""
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import time

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

print("Libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"MPS (Apple Silicon GPU) available: {torch.backends.mps.is_available()}")
print(f"MPS built: {torch.backends.mps.is_built()}")

Libraries imported successfully!
PyTorch version: 2.5.1
CUDA available: False
MPS (Apple Silicon GPU) available: True
MPS built: True


### Device Setup

In [7]:
def setup_device():
    """
    Configure the computation device for optimal performance.
    
    Device Priority:
    1. MPS (Metal Performance Shaders) - Apple Silicon GPU (M1/M2/M3)
    2. CUDA - NVIDIA GPU
    3. CPU - Fallback option
    
    Returns:
        torch.device: The device to be used for tensor operations
    """
    # Check for Apple Silicon GPU (MPS)
    if torch.backends.mps.is_available() and torch.backends.mps.is_built():
        device = torch.device("mps")
        print("\n" + "="*80)
        print("DEVICE CONFIGURATION")
        print("="*80)
        print("Using: Apple Silicon GPU (Metal Performance Shaders)")
        print("✓ M1/M2/M3/M4 GPU acceleration enabled")
        print("✓ Unified memory architecture optimized")
    # Check for NVIDIA GPU (CUDA)
    elif torch.cuda.is_available():
        device = torch.device("cuda")
        print("\n" + "="*80)
        print("DEVICE CONFIGURATION")
        print("="*80)
        print(f"Using: NVIDIA GPU - {torch.cuda.get_device_name(0)}")
        print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
        print("✓ CUDA acceleration enabled")
    # Fallback to CPU
    else:
        device = torch.device("cpu")
        print("\n" + "="*80)
        print("DEVICE CONFIGURATION")
        print("="*80)
        print("Using: CPU")
        print("⚠️ GPU acceleration not available - training will be slower")
        print("Consider using Google Colab for free GPU access")
    
    print("="*80)
    return device

# Initialize device
device = setup_device()


DEVICE CONFIGURATION
Using: Apple Silicon GPU (Metal Performance Shaders)
✓ M1/M2/M3/M4 GPU acceleration enabled
✓ Unified memory architecture optimized


### Dataset Loadin and Preprocessing 

In [8]:
def load_cifar10_data(batch_size=64, num_workers=0):
    """
    Load and preprocess CIFAR-10 dataset for ResNet-50.
    
    Dataset: 60,000 images (32×32) in 10 classes
    Preprocessing: Resize to 224×224, ImageNet normalization
    Split: 40k train / 10k validation / 10k test
    
    Args:
        batch_size (int): Samples per batch (64 optimal for M1)
        num_workers (int): Data loading threads (0 for MPS)
    
    Returns:
        tuple: (train_loader, val_loader, test_loader, class_names)
    """
    # ImageNet normalization constants
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]

    # Training augmentation
    train_transform = transforms.Compose([
        transforms.Resize(224),  # Resize to 224×224
        transforms.RandomHorizontalFlip(p=0.5),  # Random flip
        transforms.RandomCrop(224, padding=4),  # Random crop
        transforms.ColorJitter(brightness=0.2, contrast=0.2),  # Color augmentation
        transforms.ToTensor(),  # Convert to tensor
        transforms.Normalize(mean, std)  # Normalize
    ])
    # Validation/test transform (no augmentation)
    test_transform = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])
    # Download CIFAR-10
    print("Loading CIFAR-10 dataset...")
    train_val_dataset = datasets.CIFAR10(
        root='./data',
        train=True,
        download=True,
        transform=train_transform
    )
    
    test_dataset = datasets.CIFAR10(
        root='./data',
        train=False,
        download=True,
        transform=test_transform
    )

    # Split train into train/val (80/20)
    train_size = int(0.8*len(train_val_dataset))
    val_size = len(train_val_dataset) - train_size

    train_dataset, val_dataset = random_split(
        train_val_dataset,
        [train_size, val_size],
        generator=torch.Generator().manual_seed(42)
    )

    # Apply test transform to validation
    val_dataset.dataset = datasets.CIFAR10(
        root='./data',
        train=True,
        download=False,
        transform=test_transform
    )

    # Create data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=False  # Not needed for MPS
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=False
    )
    
    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=False
    )

    # Class names
    classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']
    
    # Print statistics
    print(f"\nDataset Statistics:")
    print(f"  Training samples: {len(train_dataset):,}")
    print(f"  Validation samples: {len(val_dataset):,}")
    print(f"  Test samples: {len(test_dataset):,}")
    print(f"  Number of classes: {len(classes)}")
    print(f"  Batch size: {batch_size}")
    print(f"  Image size: 224×224×3")
    print(f"  Normalization: mean={mean}, std={std}")
    
    return train_loader, val_loader, test_loader, classes
# Load CIFAR-10 data
train_loader, val_loader, test_loader, class_names = load_cifar10_data(batch_size=64, num_workers=0)

Loading CIFAR-10 dataset...


100%|██████████| 170M/170M [00:19<00:00, 8.86MB/s] 


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified

Dataset Statistics:
  Training samples: 40,000
  Validation samples: 10,000
  Test samples: 10,000
  Number of classes: 10
  Batch size: 64
  Image size: 224×224×3
  Normalization: mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
